In [2]:
import pandas as pd
from datetime import datetime

## Read in data 

### Market data

In [21]:
btc_market = pd.read_csv("data/BTC-USD-80.csv")
btc_market.head()

,time,close,high,low,open,volumefrom,volumeto,mid
0,1582984800,8645.88,8678.25,8606.31,8676.27,954.50,8238841.34,8642.280
1,1582988400,8693.73,8708.76,8619.73,8645.88,785.84,6824127.12,8664.245
2,1582992000,8638.05,8702.50,8630.15,8693.73,761.92,6629631.43,8666.325
3,1582995600,8701.21,8702.40,8623.94,8638.05,951.78,8259525.21,8663.170
4,1582999200,8695.56,8714.49,8666.65,8701.21,556.50,4862181.37,8690.570


#### Convert unix code time to more readable datetime format

In [23]:
btc_market['time_formatted'] = pd.to_datetime(btc_market['time'], unit='s')

In [25]:
btc_market[['time', 'time_formatted']].head()

,time,time_formatted
0,1582984800,2020-02-29 14:00:00
1,1582988400,2020-02-29 15:00:00
2,1582992000,2020-02-29 16:00:00
3,1582995600,2020-02-29 17:00:00
4,1582999200,2020-02-29 18:00:00


### Tweets

In [30]:
btc_tweets = pd.read_json("data/tweets_btc_march.json")
btc_tweets.head()

,has_media,hashtags,img_urls,is_replied,is_reply_to,likes,links,parent_tweet_id,replies,reply_to_users,...,screen_name,text,text_html,timestamp,timestamp_epochs,tweet_id,tweet_url,user_id,username,video_url
0,False,"[privacy, bitcoin]",[],False,False,1,[https://snip.ly/m053t8],,0,[],...,AnabellHilarski,Snowden la película que cambio mi forma de pen...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-03-06 23:59:25,2020-03-06 23:59:25,1236078965380562944,/AnabellHilarski/status/1236078965380562945,1299927211,✨Anabell Hilarski✨,
1,True,"[USD, EUR, CNY, GBP, RUBpic]",[https://pbs.twimg.com/media/ESdvPzGWkAAgtoN.png],False,False,0,[https://www.worldcoinindex.com/coin/bitcoin],,0,[],...,WorldCoinIndex,Bitcoin price index https://www.worldcoinindex...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-03-06 23:59:20,2020-03-06 23:59:20,1236078944392228864,/WorldCoinIndex/status/1236078944392228866,2395138046,WorldCoinIndex,
2,False,[],[],False,True,1,[],1235468311791464448,0,"[{'screen_name': 'Auslad', 'user_id': '1715190...",...,alang,"Mate, the internet will fall apart if the bank...","<p class=""TweetTextSize js-tweet-text tweet-te...",2020-03-06 23:59:17,2020-03-06 23:59:17,1236078931964334080,/alang/status/1236078931964334080,142963,25 million people,
3,False,[],[],True,True,2,[http://yeticold.com],1236070982630195201,1,"[{'screen_name': 'Donhobluesky', 'user_id': '5...",...,JWWeatherman_,Agreed. I’m obviously going to recommend http:...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-03-06 23:59:15,2020-03-06 23:59:15,1236078923643043840,/JWWeatherman_/status/1236078923643043840,920399561272594432,JW Weatherman | mathbot.com,
4,False,"[Bitcoin, latenightthoughts]",[],True,False,257,[],,12,[],...,IvanOnTech,The best thing about #Bitcoin is the COMMUNITY...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-03-06 23:59:13,2020-03-06 23:59:13,1236078916923723776,/IvanOnTech/status/1236078916923723776,390627208,Ivan on Tech,


In [31]:
btc_tweets.columns

Index(['has_media', 'hashtags', 'img_urls', 'is_replied', 'is_reply_to',
       'likes', 'links', 'parent_tweet_id', 'replies', 'reply_to_users',
       'retweets', 'screen_name', 'text', 'text_html', 'timestamp',
       'timestamp_epochs', 'tweet_id', 'tweet_url', 'user_id', 'username',
       'video_url'],
      dtype='object')

In [35]:
btc_tweets.sort_values(by='timestamp', inplace=True)

In [36]:
btc_tweets.head()

,has_media,hashtags,img_urls,is_replied,is_reply_to,likes,links,parent_tweet_id,replies,reply_to_users,...,screen_name,text,text_html,timestamp,timestamp_epochs,tweet_id,tweet_url,user_id,username,video_url
790,False,"[EARN, BRAVEBROWSER, crypto, cryptocurrency, C...",[],False,False,3,[https://brave.com/ace913],,0,[],...,Ace_Lolzz,#EARN $BAT ON THE #BRAVEBROWSER !\n\nA few sim...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-02-29 23:56:34,2020-02-29 23:56:34,1233903923272060928,/Ace_Lolzz/status/1233903923272060928,892760065,AceLolzz,
789,False,[],[],False,True,0,[],1231843154409730048,0,"[{'screen_name': 'bitcoin_brown', 'user_id': '...",...,Petermccarthy84,XRP?,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-02-29 23:56:37,2020-02-29 23:56:37,1233903933346762752,/Petermccarthy84/status/1233903933346762757,268474320,MrMac,
788,False,[],[],False,False,0,"[http://t.me/VRbitcoin, https://www.pscp.tv/w/...",,0,[],...,LupexNews,"Udi Wertheimer: ""Starting the VR presentation ...","<p class=""TweetTextSize js-tweet-text tweet-te...",2020-02-29 23:56:38,2020-02-29 23:56:38,1233903939986153472,/LupexNews/status/1233903939986153473,1128475195667144704,Lupex News,
787,False,"[Bitcoin, Gold]",[],True,True,2,[],1233398104659025921,1,"[{'screen_name': 'SantiagoAuFund', 'user_id': ...",...,TraderStef,"Feb. 2020 highs, #Bitcoin crushed by -20%, Gol...","<p class=""TweetTextSize js-tweet-text tweet-te...",2020-02-29 23:56:48,2020-02-29 23:56:48,1233903979945496576,/TraderStef/status/1233903979945496577,1851877248,TraderStef,
786,False,[Bitcoin],[],False,True,0,[],1233776262587412480,0,"[{'screen_name': 'percymild', 'user_id': '3627...",...,ixkillax,#Bitcoin\n\nTg:@ixkillax,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-02-29 23:56:52,2020-02-29 23:56:52,1233903998119268352,/ixkillax/status/1233903998119268353,1159278650966982656,Ixkillax,


### Merge two data frames

In [40]:
pd.merge_asof(btc_market, btc_tweets, left_on='time_formatted', right_on='timestamp', direction='forward', tolerance=pd.Timedelta('1h'))

,time,close,high,low,open,volumefrom,volumeto,mid,time_formatted,has_media,...,screen_name,text,text_html,timestamp,timestamp_epochs,tweet_id,tweet_url,user_id,username,video_url
0,1582984800,8645.88,8678.25,8606.31,8676.27,954.50,8238841.34,8642.280,2020-02-29 14:00:00,NaN,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
1,1582988400,8693.73,8708.76,8619.73,8645.88,785.84,6824127.12,8664.245,2020-02-29 15:00:00,NaN,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
2,1582992000,8638.05,8702.50,8630.15,8693.73,761.92,6629631.43,8666.325,2020-02-29 16:00:00,NaN,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
3,1582995600,8701.21,8702.40,8623.94,8638.05,951.78,8259525.21,8663.170,2020-02-29 17:00:00,NaN,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
4,1582999200,8695.56,8714.49,8666.65,8701.21,556.50,4862181.37,8690.570,2020-02-29 18:00:00,NaN,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1916,1589882400,9782.20,9800.49,9756.08,9781.42,1831.88,17917644.42,9778.285,2020-05-19 10:00:00,NaN,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
1917,1589886000,9734.17,9890.69,9698.57,9782.20,5844.63,57351450.36,9794.630,2020-05-19 11:00:00,NaN,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
1918,1589889600,9614.99,9749.27,9571.68,9734.17,6755.46,65106618.78,9660.475,2020-05-19 12:00:00,NaN,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
1919,1589893200,9664.15,9679.35,9595.02,9614.99,3405.25,32820135.94,9637.185,2020-05-19 13:00:00,NaN,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
